In [5]:
import pandas as pd
import collections
import json
from datetime import datetime
import glob, os, sys, calendar
os.environ['TZ'] = 'Asia/Calcutta'

try:
    #base_path = "D:/jupyter notebook/Hackathon/Neuro-Bot"
    #master_path2 =  os.path.dirname(os.path.realpath(__file__))
    master_path2 = os.getcwd()
    location2    = int(master_path2.find("Neuro-Bot"))
    base_path    = master_path2[0:location2] + 'Neuro-Bot'
    sys.path.insert(0, base_path)
except Exception as e:
    print(e)
    pass
base_path

'd:\\jupyter notebook\\Hackathon\\Neuro-Bot'

In [ ]:
# Data Reading
df = pd.read_json(f"{base_path}/dataset/messages.json")
df2 = df[['conversations']]

In [ ]:
# df[['conversations']].iloc[:5000].to_csv(f"{base_path}/dataset/messages_1.csv")
# df[['conversations']].iloc[5000:10000].to_csv(f"{base_path}/dataset/messages_2.csv")
# df[['conversations']].iloc[10000:].to_csv(f"{base_path}/dataset/messages_3.csv")

In [ ]:
# 1. Each row has its own unique name which present in displayName
# 2. conversationblocked is False in properties

In [ ]:
# id , displayName, version, properties.conversationblocked, MessageList

In [ ]:
df2['displayName'] = df2.conversations.apply( lambda x: x['displayName'])
df2['version'] = df2.conversations.apply( lambda x: x['version'])
df2['conversationblocked'] = df2.conversations.apply( lambda x: x['properties']['conversationblocked'])
df2['conversationstatus'] = df2.conversations.apply( lambda x: x['properties']['conversationstatus'])

#[df2[i].nunique() for i in df2.columns.tolist()]
#df2['displayName'].nunique() # 14829
#df2.version.nunique() # 16239
#df2.conversationstatus.value_counts()
#df2.head()

In [ ]:
for index in range(0,2):
    for val in df2.conversations[index].keys():
        print(val)
        if val != 'MessageList':
            print("\t", df2.conversations[index][val])
    print('----------------------------------')

In [ ]:
final_df = pd.DataFrame()
for idx, val in df2.iterrows():
    try:
        name = val.displayName
        temp = pd.DataFrame(val.conversations['MessageList'])[['displayName', 'originalarrivaltime', 'messagetype', 'content']]
        temp = temp.loc[temp.messagetype == 'RichText']
        if not len(temp) and temp.displayName.nunique() > 2:
            continue
        temp = temp.sort_values('originalarrivaltime',ascending=True)
        s = temp['displayName'].isnull()
        temp = (temp.groupby([s.cumsum(),s], sort=False)
        .agg({'displayName':'first', 'originalarrivaltime':'first', 'content': '. '.join})
        .reset_index(drop=True)
        )
        s = temp['displayName'].eq(name)
        temp = (temp.groupby([s.cumsum(),s], sort=False)
        .agg({'displayName':'first', 'originalarrivaltime':'first', 'content': '. '.join})
        .reset_index(drop=True)
        )
        temp = temp.pivot(columns='displayName', values='content')
        temp.columns = temp.columns.astype(str)
        if isinstance(temp[name].iloc[0], str):
            temp = pd.concat([temp[name], temp[['nan']].shift(-1).reset_index(drop=True)], axis=1).dropna()
        else:
            temp = pd.concat([temp['nan'], temp[[name]].shift(-1).reset_index(drop=True)], axis=1).dropna()
        temp = temp.rename({name:"question", "nan":"answer"}, axis=1)
        final_df = final_df.append(temp, ignore_index=True)
    except:
        #print(idx)
        pass
    

In [ ]:
final_df = final_df.dropna().drop_duplicates()
final_df.to_csv(f"{base_path}/dataset/processed.csv", index=False)

#### Reading saved dataframe

In [ ]:
df = pd.read_csv(f"{base_path}/dataset/processed.csv")

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:


# Single list of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

#Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))